In [16]:
import pandas as pd
import requests #helps us fetch data from api
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder #converts categorical valus into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score#to measure accuracy of our prediction
from datetime import datetime, timedelta #to handle data and time
import pytz #to handle timezones
from IPython import get_ipython
from IPython.display import display


In [17]:
API_KEY = 'OPENWEATHER_API_KEY' #from open weather map api
BASE_URL = 'hhttps://api.openweathermap.org/data/2.5/' #base url for making API requests

fetching the currebt weather data

In [26]:
import requests   # make sure this import is executed once

BASE_URL = "https://api.openweathermap.org/data/2.5/"
API_KEY  = "OPENWEATHER_API_KEY"          # ← put your real key here

def get_current_weather(city):
    # 1. build request URL  (note the '=' after appid)
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"

    # 2. call the API correctly
    response = requests.get(url)        # ← FIXED
    data = response.json()

    # 3. extract the fields your weather_view() expects
    return {
        "current_temp":     data["main"]["temp"],
        "feels_like":       data["main"]["feels_like"],
        "temp_min":         data["main"]["temp_min"],
        "temp_max":         data["main"]["temp_max"],
        "humidity":         data["main"]["humidity"],
        "pressure":         data["main"]["pressure"],
        "description":      data["weather"][0]["description"],
        "country":          data["sys"]["country"],

        # use .get so code doesn’t crash if field is missing
        "wind_gust_dir":    data["wind"].get("deg", 0),
        "wind_gust_speed":  data["wind"].get("gust", 0),
    }


read and clean historical data

In [35]:
def read_historical_data(file_path):
  df = pd.read_csv('/content/weather.csv')
  df = df.dropna()
  df = df.drop_duplicates()
  return df
  df = df.drop_duplicates
  return df
  df = df.drop_duplicates()
  return df
  df = df.drop_duplicates()
  return df

In [36]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  x = data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']]
  y = data['RainTomorrow']
  return x,y,le

In [37]:
def train_rain_model(x,y):
  x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(x_train,y_train)
  y_pred = model.predict(x_test)
  print("mean squared error for rain model")
  print(mean_squared_error(y_test,y_pred))
  return model

In [46]:
def prepare_regression_data(data, feature):
  x,y = [], []
  # The loop should go up to the second-to-last element (len(data) - 1)
  # to predict the next element at index i + 1.
  # len(data) gives the number of rows. The last index is len(data) - 1.
  # We iterate up to len(data) - 1 to access index i and i+1.
  for i in range(len(data) - 1):
    x.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])
  x = np.array(x).reshape(-1,1)
  y = np.array(y)
  return x, y

In [47]:
def train_regression_model(x,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(x,y)
  return model

predicitng future

In [48]:
def predict_future(model,current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array(predictions[-1]).reshape(1,-1))
    predictions.append(next_value[0])
  return predictions[1:]

weather analysis function

In [57]:
import pandas as pd
from datetime import datetime, timedelta
import pytz

# ───────────────────────────────────────────────
#  WEATHER-VIEW  (cleaned-up version)
# ───────────────────────────────────────────────
def weather_view():
    # 1)  Ask for a city and pull current weather
    # -------------------------------------------------
    city = input("Enter any city name : ").strip()
    current_weather = get_current_weather(city)                 # ← your helper
    historical_data = read_historical_data("/content/sonar data.csv")

    # 2)  Train the rain-/label encoder model
    # -------------------------------------------------
    X, y, le = prepare_data(historical_data)
    rain_model = train_rain_model(X, y)

    # 3)  Convert wind degrees → compass point
    # -------------------------------------------------
    wind_deg = current_weather["wind_gust_dir"] % 360
    compass_points = [
        ("N",   348.75, 360.0), ("N",   0.0,   11.25),
        ("NNE", 11.25,  33.75), ("NE",  33.75, 56.25),
        ("ENE", 56.25,  78.75), ("E",   78.75,101.25),
        ("ESE",101.25, 123.75), ("SE", 123.75,146.25),
        ("SSE",146.25, 168.75), ("S",  168.75,191.25),
        ("SSW",191.25, 213.75), ("SW", 213.75,236.25),
        ("WSW",236.25, 258.75), ("W",  258.75,281.25),
        ("WNW",281.25, 303.75), ("NW", 303.75,326.25),
        ("NNW",326.25, 348.75),
    ]
    compass_direction = next(pt for pt, start, end in compass_points
                              if start <= wind_deg < end)
    compass_direction_encoded = (
        le.transform([compass_direction])[0]
        if compass_direction in le.classes_
        else -1
    )

    # 4)  Build the single-row “current” dataframe
    # -------------------------------------------------
    current_row = {
        "MinTemp":        current_weather["temp_min"],
        "MaxTemp":        current_weather["temp_max"],
        "WindGustDir":    compass_direction_encoded,
        "WindGustSpeed":  current_weather["wind_gust_speed"],  # note lowercase key
        "Humidity":       current_weather["humidity"],
        "Pressure":       current_weather["pressure"],
        "Temp":           current_weather["current_temp"],
    }
    rain_prediction = bool(rain_model.predict(pd.DataFrame([current_row]))[0])

    # 5)  Regression models for future temp & humidity
    # -------------------------------------------------
    X_temp, y_temp = prepare_regression_data(historical_data, "Temp")
    X_hum,  y_hum  = prepare_regression_data(historical_data, "Humidity")
    temp_model = train_regression_model(X_temp, y_temp)
    hum_model  = train_regression_model(X_hum,  y_hum)

    future_temp     = predict_future(temp_model, current_weather["temp_min"])
    future_humidity = predict_future(hum_model,  current_weather["humidity"])

    # 6)  Build labels for the next 5 whole hours
    # -------------------------------------------------
    tz = pytz.timezone("Asia/Karachi")
    now = datetime.now(tz)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # 7)  Pretty-print everything
    # -------------------------------------------------
    print(f"\nCity                : {city}, {current_weather['country']}")
    print(f"Current Temperature : {current_weather['current_temp']} °C")
    print(f"Feels Like          : {current_weather['feels_like']} °C")
    print(f"Min Temperature     : {current_weather['temp_min']} °C")
    print(f"Max Temperature     : {current_weather['temp_max']} °C")
    print(f"Humidity            : {current_weather['humidity']} %")
    print(f"Weather Condition   : {current_weather['description']}")
    print(f"Rain Expected?      : {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Predictions:")
    for t, temp in zip(future_times, future_temp):
        print(f"{t}: {round(temp,1)} °C")

    print("\nFuture Humidity Predictions:")
    for t, hum in zip(future_times, future_humidity):
        print(f"{t}: {round(hum,1)} %")


weather_view()

Enter any city name : malappuram
mean squared error for rain model
0.1506849315068493

City                : malappuram, IN
Current Temperature : 29.09 °C
Feels Like          : 34.77 °C
Min Temperature     : 29.09 °C
Max Temperature     : 29.09 °C
Humidity            : 79 %
Weather Condition   : broken clouds
Rain Expected?      : Yes

Future Temperature Predictions:
12:00: 21.8 °C
13:00: 21.3 °C
14:00: 19.8 °C
15:00: 18.2 °C
16:00: 16.8 °C

Future Humidity Predictions:
12:00: 54.0 %
13:00: 58.1 %
14:00: 54.2 %
15:00: 58.1 %
16:00: 54.2 %
